In [2]:
from bs4 import BeautifulSoup
import requests

url = 'https://www.eprbulletsafsc.com/'
page = requests.get(url)
soup = BeautifulSoup(page.content, 'html.parser')

In [56]:
from collections import defaultdict


anchor_tags = soup.find_all('a')
required_urls = defaultdict(list)
for tag in anchor_tags:
    href = tag['href']
    category = tag.text.split(' ')[0]

    cat_prefix = category[0] + 'XXXX'

    linked_page_url = url + "/" + href
    # Only consider one general category
    if not category[0].isdigit(): 
        required_urls['General'].append(linked_page_url)
    else:
        required_urls[cat_prefix].append(linked_page_url)

print(required_urls)

defaultdict(<class 'list'>, {'1XXXX': ['https://www.eprbulletsafsc.com//epr_1A0X1.htm', 'https://www.eprbulletsafsc.com//epr_1A1X1.htm', 'https://www.eprbulletsafsc.com//epr_1A2X1.htm', 'https://www.eprbulletsafsc.com//epr_1A3X1.htm', 'https://www.eprbulletsafsc.com//epr_1A4X1.htm', 'https://www.eprbulletsafsc.com//epr_1A6X1.htm', 'https://www.eprbulletsafsc.com//epr_1A7X1.htm', 'https://www.eprbulletsafsc.com//epr_1A8X1.htm', 'https://www.eprbulletsafsc.com//epr_1B4X1.htm', 'https://www.eprbulletsafsc.com//epr_1C0X2.htm', 'https://www.eprbulletsafsc.com//epr_1C1X1.htm', 'https://www.eprbulletsafsc.com//epr-1C3X1.htm', 'https://www.eprbulletsafsc.com//epr_1C4X1.htm', 'https://www.eprbulletsafsc.com//epr_1C5X1.htm', 'https://www.eprbulletsafsc.com//epr_1C6X1.htm', 'https://www.eprbulletsafsc.com//epr_1C7X1.htm', 'https://www.eprbulletsafsc.com//epr_1C8X1.htm', 'https://www.eprbulletsafsc.com//epr_1C8X2.htm', 'https://www.eprbulletsafsc.com//epr_1C8X3.htm', 'https://www.eprbulletsafsc.co

In [58]:
res = {}
for category in required_urls:
    linked_page_urls = required_urls[category]
    if category in res: continue # Prevent Overwriting

    res[category] = []

    for linked_page_url in linked_page_urls:
        linked_page = requests.get(linked_page_url)
        linked_soup = BeautifulSoup(linked_page.content, 'html.parser')
        divSoup = linked_soup.find("div", {"class" : "main"})
        ps = divSoup.findAll('p')

        for p in ps:
            splits = p.text.split('\n')

            for line in splits:
                if line != '' and line[2:] not in res[category]:
                    res[category].append(line[2:])
                
                if len(res[category]) == 20: break

            if len(res[category]) == 20: break

        if len(res[category]) == 20: break

print(res)

{'1XXXX': ['#1/10 Basic Boom Operator Course; maintained stellar 98% avg grade--earned Distinguished Graduate award ', '100K+ lbs fuel to 12 redeploying F-16s during critical overwater refueling--combat acft returned home safely  ', "1st Class aviator--offloaded 110K lbs of fuel to 7 F-15 s; spt'd AOR Coronet; weapon sys's in combat theater ", '1st class job performance; created tracking sys/built 110 EPR/OPR folders; 100% accountability of sq reports ', '340 EARS "Boom/Aircrew/Mo" (#1 of 42); spt\'d 50 combat msns-supported Sq ops w/98% effectiveness rate ', "351 EARS NCO OTM; key to Wg Sched's OSS Tm OTY/led 5 morale events for 58 dply'd Amn--TSgt a must! ", '6 OSS NCO Professional Performer OTQ 4Q11; my guru in Wg scheduling for AMC level rqmts--TSgt ASAP ', '618 AOC Holiday Party cmte mbr; dir POC for ticket sales--1st-class event prepared for 500-mbr org to enjoy ', '91 ARS Amn Professional Performer OTQ; established example and work ethic crucial to Sq function and msn ', 'Academ

In [60]:
import pandas as pd

for key in res:
    print(len(res[key]))

df = pd.DataFrame(res.items(), columns=['Category', 'Data']).explode('Data').reset_index(drop=True)
print(df)
df.to_excel('output.xlsx', index=False)

20
20
20
20
20
20
11
20
20
20
    Category                                               Data
0      1XXXX  #1/10 Basic Boom Operator Course; maintained s...
1      1XXXX  100K+ lbs fuel to 12 redeploying F-16s during ...
2      1XXXX  1st Class aviator--offloaded 110K lbs of fuel ...
3      1XXXX  1st class job performance; created tracking sy...
4      1XXXX  340 EARS "Boom/Aircrew/Mo" (#1 of 42); spt'd 5...
..       ...                                                ...
186  General  Dedicated wingman; volunteered 13 hrs to Sober...
187  General  Dedicated XX hrs to AADD pgm; ensured sfty of ...
188  General  Designated driver for AADD program; donated 12...
189  General  Designated driver for Combat Dining In; escort...
190  General  Designated driver for MXS Christmas party; pro...

[191 rows x 2 columns]
